In [ ]:
import numpy as np
from scipy.signal import find_peaks

def find_prominent_peaks(magnitude_spectrum, threshold, prominence):
    # Find all peaks in the magnitude spectrum
    peaks, _ = find_peaks(magnitude_spectrum, height=threshold, prominence=prominence)
    # Filter out peaks at negative frequencies
    #positive_peaks = [peak for peak in peaks if peak > len(magnitude_spectrum) // 2]
    # Sort the peaks by their prominence
    sorted_peaks = sorted(peaks, key=lambda p: -magnitude_spectrum[p])
    # Calculate the cumulative sum of the magnitude spectrum
    cumulative_sum = np.cumsum(magnitude_spectrum[sorted_peaks])
    # Find the number of peaks that make up at least 90% of the total data
    total_sum = np.sum(magnitude_spectrum[sorted_peaks])
    n_peaks = np.searchsorted(cumulative_sum, 0.2 * total_sum, side='right')
    return sorted_peaks[:n_peaks]

#Find the <k,w> values 
from numba import jit

def find_prominent_freqs(magnitude_spectrum, sample_rate, threshold=0.1, prominence=0.1):
    # Find all peaks in the magnitude spectrum
    peaks, _ = find_peaks(magnitude_spectrum, height=threshold, prominence=prominence)
    # Sort the peaks by their prominence
    sorted_peaks = sorted(peaks, key=lambda p: -magnitude_spectrum[p])
    # Calculate the frequency bins
    freq_bins = np.fft.fftfreq(len(magnitude_spectrum), 1 / sample_rate)
    # Get the frequency values corresponding to the prominent peaks
    freq_values = freq_bins[sorted_peaks]
    return freq_values

def find_kw(magnitude_spectrum):
    pks=find_prominent_peaks(magnitude_spectrum, threshold=0.1, prominence=0.1)
    fr=find_prominent_freqs(magnitude_spectrum_1, sample_rate, threshold=0.1, prominence=0.1)
    n = len(fr)
    subtraction_array = []
    for i in range(n):
        for j in range(i + 1, n):
            diff = (fr[i] - fr[j])
            subtraction_array.append(diff)
    return subtraction_array

def gamma_function(magnitude_spectrum,threshold, prominence, subtraction_array,x):
    summation=[]
    d=len(find_prominent_peaks(magnitude_spectrum, threshold, prominence))
    i=1
    for i in range(d):
        exponentials = np.exp(subtraction_array * x * 1j)*i
        result = np.sum(exponentials)
        summation.append(result)
    func=np.sum(summation)+1
    return func

def optimal_tau(x,magnitude_spectrum,threshold, prominence, subtraction_array):
    x0=0
    x_vls = np.linspace(0, np.pi, 1000)
    gamma_values = [gamma_function(magnitude_spectrum, threshold, prominence, subtraction_array, x) for x in x_values]
    min_index = np.argmin(gamma_values)
    o_tau = x_vls[min_index]
    return o_tau
    

